In [107]:
from surrealdb import Surreal
import asyncio
import pandas as pd
import json
import yaml
import openai
from tqdm import tqdm

In [108]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [109]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")

  df = pd.read_csv("drugs/trials_umls.tsv", sep="\t", lineterminator="\n")

  for index, row in df.iterrows():
    res = await db.create(
      "Trial",
      {
        "id": row["Posting ID"],
        "Sponsor": row["Sponsor"],
        "StudyTitle": row["Study Title"].replace(":", "\:"),
        "Drug": row["Medicine or Vaccine (generic name)"],
        "Disorder": row["Medical Condition"],
        "Phase": row["Phase"],
        "LinkToSponsorStudyRegistry": row["Link to Sponsor Study Registry"],
        "LinkToClinicalTrials": row["Link to study details on ClinicalTrials.gov (if available)"],
        "cui": [f"Drug:{x}" for x in row["CUI_umls"].split(";")],
        "PreferredUMLSName": row["preferred_umls_name"].split(";"),
      },
    )

    for x in row["CUI_umls"].split(";"):
      res = await db.query(f"RELATE Trial:{row['Posting ID']}->TESTS->Drug:{x};")


In [110]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")

  res = await db.query(
    "DEFINE ANALYZER custom_analyzer TOKENIZERS blank FILTERS lowercase, snowball(english);"
  )
  print(res)
  
  res = await db.query(
    "DEFINE INDEX study_title ON Trial FIELDS StudyTitle SEARCH ANALYZER custom_analyzer BM25 HIGHLIGHTS;"
  )
  print(res)

[{'result': None, 'status': 'OK', 'time': '328.958µs'}]
[{'result': None, 'status': 'OK', 'time': '5.248049877s'}]


In [111]:
async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")
    res = await db.query("SELECT * FROM Trial WHERE StudyTitle @@ 'double blind' LIMIT 1;")
    print(res)

[{'result': [{'Disorder': 'Asthma', 'Drug': 'fluticasone furoate', 'LinkToClinicalTrials': 'http://clinicaltrials.gov/show/NCT00766090', 'LinkToSponsorStudyRegistry': 'https://www.gsk-studyregister.com/en/trial-details/?id=112202', 'Phase': 'Phase 2', 'PreferredUMLSName': ['fluticasone furoate'], 'Sponsor': 'GSK', 'StudyTitle': 'A Multi-Centre, Randomized, Double Blind Cross-over study to assess the non-inferiority of GW685698X 200mcg Once Daily and 100mcg Twice Daily in Adult and Adolescent Patients with Asthma', 'cui': ['Drug:C1948374'], 'id': 'Trial:383'}], 'status': 'OK', 'time': '775.459µs'}]


In [112]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")
  
  for line in open("drugs/disease_4_embedding.json", "r"):
    value = json.loads(line)

    res = await db.create(
      "Disorder",
      {
        "id": value["cui"], 
        "name": value["name"], 
        "definition": value.get("definition", ""),
        "definitionEmbedding": value["definitionEmbedding"]
      }
    )

In [113]:
async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    res = await db.query("DEFINE INDEX disorder_index ON TABLE Disorder COLUMNS id UNIQUE;")
    print (res)

    res = await db.query("DEFINE INDEX idx_embedding ON Disorder FIELDS definitionEmbedding MTREE DIMENSION 1536 DIST COSINE;")
    print (res)

[{'result': None, 'status': 'OK', 'time': '179.374291ms'}]
[{'result': None, 'status': 'OK', 'time': '12.200820422s'}]


In [114]:
question = "give me some joint-related disorders"

question_embedding = get_embedding(question)

#-- Select actors who look like you and have won an Oscar
#SELECT id, flag, vector::distance::knn() AS distance FROM actor WHERE flag = true AND embedding <|2,40|> $person_embedding ORDER BY distance;

async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"SELECT name, definition FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|5|> {question_embedding};")
    print (res)
    #await db.use("drug_namespace", "drug_trial")

    #res = await db.query(f"SELECT name, definition, vector::distance::knn() AS distance FROM disorder WHERE definition != 'undefined' AND definitionEmbedding <|1536,40|> {question_embedding} ORDER BY distance;")
    #print (res)



[{'result': [{'definition': 'Diseases involving the JOINTS.', 'name': 'Arthropathy'}, {'definition': 'Pain in the joint.', 'name': 'Arthralgia'}, {'definition': 'Disorders of connective tissue, especially the joints and related structures, characterized by inflammation, degeneration, or metabolic derangement.', 'name': 'Rheumatism'}, {'definition': 'Acute or chronic inflammation of JOINTS.', 'name': 'Arthritis'}, {'definition': 'Heterogeneous group of arthritic diseases sharing clinical and radiologic features. They are associated with the HLA-B27 ANTIGEN and some with a triggering infection. Most involve the axial joints in the SPINE, particularly the SACROILIAC JOINT, but can also involve asymmetric peripheral joints. Subsets include ANKYLOSING SPONDYLITIS; REACTIVE ARTHRITIS; PSORIATIC ARTHRITIS; and others.', 'name': 'Spondylarthropathies'}], 'status': 'OK', 'time': '13.493625ms'}]


In [115]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")
  
  for line in open("drugs/drugs_4.json", "r"):
    value = json.loads(line)

    res = await db.create(
      "Drug",
      {
        "id": value["cui"], 
        "name": value["name"]
      }
    )

In [116]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")

  res = await db.query("DEFINE INDEX drug_index ON TABLE Drug COLUMNS id UNIQUE;")
  print (res)


[{'result': None, 'status': 'OK', 'time': '55.912375ms'}]


In [117]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")
  df = pd.read_csv("drugs/moa_3.tsv", sep='\t', lineterminator='\n')

  for index, row in df.iterrows():
    res = await db.create(
      "MOA",
      {
        "id": row["id"], 
        "name": row["name"]
      }
    )

In [118]:
async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    res = await db.query("DEFINE INDEX moa_index ON TABLE MOA COLUMNS id UNIQUE;")
    print (res)

[{'result': None, 'status': 'OK', 'time': '5.728ms'}]


In [119]:
async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")
    df = pd.read_csv("drugs/cui_indication_3.tsv", sep='\t', lineterminator='\n')

    for index, row in tqdm(df.iterrows()):
        
        # await db.query(f"LET $from_id = SELECT id from drug where cui={row['from']};")
        # #print (row['from'], from_id, "hello")
        # await db.query(f"LET $to_id = SELECT id from disorder where cui={row['to']};")
        #print (row['to'], to_id, "hello")
        #print ({'from_id': row['from'], 'to_id': row['to']})
        res = await db.query(f"RELATE Drug:{row['from']}->MAY_TREAT->Disorder:{row['to']};")
        #print (res)


8405it [00:05, 1539.24it/s]


In [120]:
async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")
    df = pd.read_csv("drugs/cui_moa_3.tsv", sep='\t', lineterminator='\n')

    for index, row in tqdm(df.iterrows()):
        #from_id = await db.query("SELECT id from drug where cui=$from", {'from': row['from']})
        #print (from_id.get(0))
        #print (from_id[0]['result'][0]["id"])
        #to_id = await db.query("SELECT id from MOA where moa_id=$to", {'to': row['to']})
        res = await db.query(f"RELATE Drug:{row['from']}->HAS_MOA->MOA:{row['to']};")

4177it [00:02, 1546.67it/s]


In [121]:
question = "joint-related disease"

question_embedding = get_embedding(question)

#-- Select actors who look like you and have won an Oscar
#SELECT id, flag, vector::distance::knn() AS distance FROM actor WHERE flag = true AND embedding <|2,40|> $person_embedding ORDER BY distance;

async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""SELECT VALUE name
                         FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|5|> {question_embedding};""")
    print (res)

[{'result': ['Arthropathy', 'Spondylarthropathies', 'Rheumatoid Arthritis', 'Ankylosing spondylitis', 'Arthralgia'], 'status': 'OK', 'time': '16.68575ms'}]


In [122]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        LET $target_diseases = SELECT name, definition, vector::similarity::cosine(definitionEmbedding, {question_embedding}) AS distance 
                                FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|3|> {question_embedding};
        RETURN $target_diseases.name;
    """)
    print (res)


[{'result': None, 'status': 'OK', 'time': '15.7265ms'}, {'result': ['Arthropathy', 'Spondylarthropathies', 'Rheumatoid Arthritis'], 'status': 'OK', 'time': '78.334µs'}]


In [123]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        LET $target_diseases = SELECT VALUE name FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|5|> {question_embedding};

        SELECT name AS disease_name, <-MAY_TREAT<-Drug.name AS drug_name FROM Disorder WHERE name IN $target_diseases
    """)
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                print (sub)


{'disease_name': 'Arthralgia', 'drug_name': []}
{'disease_name': 'Rheumatoid Arthritis', 'drug_name': ['enalapril', 'sulindac', 'adalimumab-atto', 'sodium aurothiomalate', 'ibuprofen', 'hydroxychloroquine sulfate', 'auranofin', 'valdecoxib', 'ketoprofen', 'golimumab', 'chloroquine', 'tofacitinib', 'fenoprofen', 'tofacitinib citrate', 'etanercept', 'choline salicylate', 'acetylsalicylic acid', 'celecoxib', 'tolmetin', 'ciclosporin', 'aceclofenac', 'penicillamine', 'azathioprine', 'diclofenac sodium', 'indometacin', 'aurothioglucose', 'thalidomide', 'rituximab', 'baricitinib', 'leflunomide', 'methotrexate', 'diclofenac potassium', 'enalapril maleate', 'diclofenac', 'certolizumab pegol', 'piroxicam', 'naproxen', 'adalimumab', 'flurbiprofen', 'sulfasalazine', 'upadacitinib', 'meloxicam', 'sarilumab', 'hydroxychloroquine', 'methotrexate sodium', 'anakinra', 'abatacept', 'diflunisal', 'infliximab', 'etodolac', 'nabumetone', 'tocilizumab', 'captopril', 'fish oils', 'oxaprozin']}
{'disease_nam

In [124]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        SELECT VALUE name FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|3|> {question_embedding}

    """)
    print (res)
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                print (sub)


[{'result': ['Arthropathy', 'Spondylarthropathies', 'Rheumatoid Arthritis'], 'status': 'OK', 'time': '13.69275ms'}]
Arthropathy
Spondylarthropathies
Rheumatoid Arthritis


In [125]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        LET $target_diseases = SELECT VALUE name FROM Disorder WHERE definition != 'undefined' AND definitionEmbedding <|5|> {question_embedding};

        SELECT name AS disease_name, <-MAY_TREAT<-Drug<-TESTS<-Trial.id AS trial_name FROM Disorder WHERE name IN $target_diseases
    """)
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                print (sub)

{'disease_name': 'Arthralgia', 'trial_name': []}
{'disease_name': 'Rheumatoid Arthritis', 'trial_name': ['Trial:4005', 'Trial:4775', 'Trial:4774', 'Trial:4829', 'Trial:3485', 'Trial:3492', 'Trial:3476', 'Trial:3489', 'Trial:3482', 'Trial:122', 'Trial:3481', 'Trial:3493', 'Trial:3495', 'Trial:3494', 'Trial:123', 'Trial:3483', 'Trial:3475', 'Trial:3484', 'Trial:3491', 'Trial:3496', 'Trial:3487', 'Trial:3488', 'Trial:3477', 'Trial:3486', 'Trial:3490', 'Trial:3480', 'Trial:3474', 'Trial:121', 'Trial:3479', 'Trial:3478', 'Trial:19531', 'Trial:3610']}
{'disease_name': 'Arthropathy', 'trial_name': []}
{'disease_name': 'Ankylosing spondylitis', 'trial_name': ['Trial:4829', 'Trial:3485', 'Trial:3492', 'Trial:3476', 'Trial:3489', 'Trial:3482', 'Trial:122', 'Trial:3481', 'Trial:3493', 'Trial:3495', 'Trial:3494', 'Trial:123', 'Trial:3483', 'Trial:3475', 'Trial:3484', 'Trial:3491', 'Trial:3496', 'Trial:3487', 'Trial:3488', 'Trial:3477', 'Trial:3486', 'Trial:3490', 'Trial:3480', 'Trial:3474', 'Trial

In [126]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        LET $target_diseases = SELECT VALUE name 
          FROM Disorder 
          WHERE definition != 'undefined' 
            AND definitionEmbedding <|5|> {question_embedding};

        LET $target_diseases_drugs = SELECT name 
          AS disease_name, <-MAY_TREAT<-Drug AS drug_name 
          FROM Disorder WHERE name IN $target_diseases SPLIT drug_name;

        SELECT StudyTitle, cui
        FROM Trial 
        WHERE array::intersect($target_diseases_drugs.drug_name, cui) != []

    """)
    #print (res)
    n = 0
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                print (sub)
                n += 1
    print (n)

{'StudyTitle': 'A Randomized, Double-blind, Double-dummy, Placebo-controlled, Crossover Study to Evaluate the Efficacy of TREXIMET® (Sumatriptan + Naproxen Sodium) vs. Butalbital-containing Combination Medications for the Acute Treatment of Migraine when administered during the Moderate-Severe Migraine Pain, 1 of 2', 'cui': ['Drug:C0075632', 'Drug:C0027396']}
{'StudyTitle': 'A Crossover, Randomized, Double-blind, Double-dummy, Placebo-controlled Study to Evaluate the Efficacy of TREXIMET®/Sumatriptan + Naproxen Sodium vs. Butalbital-containing Combination Medications for the Acute Treatment of Migraine when administered during Moderate-Severe Migraine Pain, 2 of 2', 'cui': ['Drug:C0075632', 'Drug:C0027396']}
{'StudyTitle': 'Assessment of the effect of sumatriptan and naproxen sodium combination tablet, sumatriptan tablet, and naproxen sodium tablet treatment on blood pressure when administered intermittently for six months for the acute treatment of migraine attacks, with or without au

In [127]:
## this one has duplicate trials

question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
LET $target_diseases = ["Rheumatoid Arthritis", "Ankylosing spondylitis"];

LET $target_diseases_drug = SELECT name AS disease_name, <-MAY_TREAT<-Drug AS drug_name FROM Disorder WHERE name IN $target_diseases;


SELECT StudyTitle, cui, $target_diseases_drug.drug_name AS drug_name, $target_diseases_drug.disease_name AS disease_name
FROM (SELECT *, (SELECT * FROM $target_diseases_drug) AS disease_drug FROM Trial SPLIT disease_drug)
WHERE array::intersect(disease_drug.drug_name, cui) != []

    """)
    #print (res)
    n = 0
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                n += 1
                print (sub)

    print (n)

{'StudyTitle': 'A Randomized, Double-blind, Double-dummy, Placebo-controlled, Crossover Study to Evaluate the Efficacy of TREXIMET® (Sumatriptan + Naproxen Sodium) vs. Butalbital-containing Combination Medications for the Acute Treatment of Migraine when administered during the Moderate-Severe Migraine Pain, 1 of 2', 'cui': ['Drug:C0075632', 'Drug:C0027396'], 'disease_name': ['Rheumatoid Arthritis', 'Ankylosing spondylitis'], 'drug_name': [['Drug:C0014025', 'Drug:C0038792', 'Drug:C4535390', 'Drug:C0018034', 'Drug:C0020740', 'Drug:C0596007', 'Drug:C0004320', 'Drug:C0913246', 'Drug:C0022635', 'Drug:C2353893', 'Drug:C0008269', 'Drug:C2930696', 'Drug:C0015837', 'Drug:C2987431', 'Drug:C0717758', 'Drug:C0055573', 'Drug:C0004057', 'Drug:C0538927', 'Drug:C0040377', 'Drug:C0010592', 'Drug:C0050403', 'Drug:C0030817', 'Drug:C0004482', 'Drug:C0700583', 'Drug:C0021246', 'Drug:C0018033', 'Drug:C0039736', 'Drug:C0393022', 'Drug:C4044947', 'Drug:C0063041', 'Drug:C0025677', 'Drug:C0282131', 'Drug:C0700

In [128]:
async with Surreal("ws://localhost:8000/rpc") as db:
  await db.signin({"user": "root", "pass": "root"})
  await db.use("drug_namespace", "drug_trial")

  res = await db.query(f"""SELECT id, name 
    FROM Drug WHERE ->MAY_TREAT->(Disorder WHERE name = "Alzheimer's Disease")""")
  
  for r in res:
    if "result" in r and r["result"] is not None and len(r["result"]) > 0:
      for sub in r["result"]:
          print (sub)

{'id': 'Drug:C0016967', 'name': 'galantamine'}
{'id': 'Drug:C0025242', 'name': 'memantine'}
{'id': 'Drug:C0036579', 'name': 'selegiline'}
{'id': 'Drug:C0039245', 'name': 'tacrine'}
{'id': 'Drug:C0042291', 'name': 'valproic acid'}
{'id': 'Drug:C0042874', 'name': 'tocopherol'}
{'id': 'Drug:C0078373', 'name': 'tocopherol acetate'}
{'id': 'Drug:C0078375', 'name': 'alpha-tocopherol succinate'}
{'id': 'Drug:C0527316', 'name': 'donepezil'}
{'id': 'Drug:C0649350', 'name': 'rivastigmine'}
{'id': 'Drug:C0771848', 'name': 'donepezil hydrochloride'}
{'id': 'Drug:C0771988', 'name': 'memantine hydrochloride'}
{'id': 'Drug:C0772415', 'name': 'rivastigmine tartrate'}
{'id': 'Drug:C0949312', 'name': 'galantamine hydrobromide'}
{'id': 'Drug:C0994472', 'name': 'dl-alpha tocopheryl acetate'}
{'id': 'Drug:C1323100', 'name': 'tocopherol,DL-alpha'}
{'id': 'Drug:C2697593', 'name': 'd-alpha-tocopheryl acetate'}
{'id': 'Drug:C2697594', 'name': 'alpha-tocopherol succinate, D-'}
{'id': 'Drug:C4043101', 'name': 'a

In [129]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
      LET $target_diseases = SELECT VALUE name 
          FROM Disorder 
          WHERE definition != 'undefined' 
            AND definitionEmbedding <|5|> {question_embedding};

      LET $target_diseases_drugs = SELECT name 
        AS disease_name, <-MAY_TREAT<-Drug AS drug_name 
        FROM Disorder WHERE name IN $target_diseases SPLIT drug_name;

      SELECT id
        FROM Trial 
        WHERE array::intersect($target_diseases_drugs.drug_name, cui) != []

    """)
    #print (res)
    n = 0
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                print (sub)
                n += 1
    print (n)

{'id': 'Trial:121'}
{'id': 'Trial:122'}
{'id': 'Trial:123'}
{'id': 'Trial:3474'}
{'id': 'Trial:3475'}
{'id': 'Trial:3476'}
{'id': 'Trial:3477'}
{'id': 'Trial:3478'}
{'id': 'Trial:3479'}
{'id': 'Trial:3480'}
{'id': 'Trial:3481'}
{'id': 'Trial:3482'}
{'id': 'Trial:3483'}
{'id': 'Trial:3484'}
{'id': 'Trial:3485'}
{'id': 'Trial:3486'}
{'id': 'Trial:3487'}
{'id': 'Trial:3488'}
{'id': 'Trial:3489'}
{'id': 'Trial:3490'}
{'id': 'Trial:3491'}
{'id': 'Trial:3492'}
{'id': 'Trial:3493'}
{'id': 'Trial:3494'}
{'id': 'Trial:3495'}
{'id': 'Trial:3496'}
{'id': 'Trial:3610'}
{'id': 'Trial:3953'}
{'id': 'Trial:3955'}
{'id': 'Trial:3956'}
{'id': 'Trial:3957'}
{'id': 'Trial:4005'}
{'id': 'Trial:4068'}
{'id': 'Trial:4069'}
{'id': 'Trial:4070'}
{'id': 'Trial:4071'}
{'id': 'Trial:4183'}
{'id': 'Trial:4184'}
{'id': 'Trial:4774'}
{'id': 'Trial:4775'}
{'id': 'Trial:4829'}
{'id': 'Trial:5006'}
{'id': 'Trial:19531'}
{'id': 'Trial:19688'}
{'id': 'Trial:20264'}
{'id': 'Trial:20265'}
{'id': 'Trial:20266'}
{'id': 'Tri

In [130]:
question = "joint-related disease"

question_embedding = get_embedding(question)


async with Surreal("ws://localhost:8000/rpc") as db:
    await db.signin({"user": "root", "pass": "root"})
    await db.use("drug_namespace", "drug_trial")

    #await db.query(f"LET $question_embedding ={question_embedding};")

    res = await db.query(f"""
        LET $target_diseases = SELECT VALUE id 
          FROM Disorder 
          WHERE definition != 'undefined' 
            AND definitionEmbedding <|5|> {question_embedding};

        SELECT name AS disease_name, <-MAY_TREAT<-Drug<-TESTS<-Trial AS trials FROM Disorder WHERE id IN $target_diseases.id

    """)
    #print (res)
    n = 0
    for r in res:
        if "result" in r and r["result"] is not None and len(r["result"]) > 0:
            for sub in r["result"]:
                if len(sub["trials"]) > 0:
                  print (sub)
                  n += 1

{'disease_name': 'Rheumatoid Arthritis', 'trials': ['Trial:4005', 'Trial:4775', 'Trial:4774', 'Trial:4829', 'Trial:3485', 'Trial:3492', 'Trial:3476', 'Trial:3489', 'Trial:3482', 'Trial:122', 'Trial:3481', 'Trial:3493', 'Trial:3495', 'Trial:3494', 'Trial:123', 'Trial:3483', 'Trial:3475', 'Trial:3484', 'Trial:3491', 'Trial:3496', 'Trial:3487', 'Trial:3488', 'Trial:3477', 'Trial:3486', 'Trial:3490', 'Trial:3480', 'Trial:3474', 'Trial:121', 'Trial:3479', 'Trial:3478', 'Trial:19531', 'Trial:3610']}
{'disease_name': 'Ankylosing spondylitis', 'trials': ['Trial:4829', 'Trial:3485', 'Trial:3492', 'Trial:3476', 'Trial:3489', 'Trial:3482', 'Trial:122', 'Trial:3481', 'Trial:3493', 'Trial:3495', 'Trial:3494', 'Trial:123', 'Trial:3483', 'Trial:3475', 'Trial:3484', 'Trial:3491', 'Trial:3496', 'Trial:3487', 'Trial:3488', 'Trial:3477', 'Trial:3486', 'Trial:3490', 'Trial:3480', 'Trial:3474', 'Trial:121', 'Trial:3479', 'Trial:3478', 'Trial:21027', 'Trial:21021', 'Trial:21018', 'Trial:21025', 'Trial:21069